## Notebook 9: Climate Trends from Reanalysis Data

**Is Missouri getting warmer? Wetter? Let's ask 40 years of data.**

Weather varies from year to year, but climate is the long-term pattern. To see if climate is changing, we need consistent measurements spanning decades. The problem? Weather stations come and go, change instruments, and don't cover everywhere.

The solution is **reanalysis**—combining all available observations (weather stations, satellites, radiosondes, ships) with physics-based atmospheric models to create a complete, consistent picture of past weather.

In this notebook, we'll:

- Use ERA5 reanalysis data to examine 40+ years of climate
- Map how summer temperatures have changed (1980s vs recent)
- Plot time series of annual temperature and precipitation for Missouri
- Calculate the warming rate in degrees per decade
- Explore how seasons are shifting

## Setup

Same initialization as previous notebooks, plus matplotlib for time series plots.

In [ ]:
%pip install -q geemap folium matplotlib

In [ ]:
import ee
from google.cloud import storage
import matplotlib.pyplot as plt
import numpy as np

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

## What is reanalysis?

**Reanalysis** is like creating a complete weather diary for the entire planet, going back decades.

**How it works:**
1. Gather ALL available observations: weather stations, satellites, ships, aircraft, radiosondes (weather balloons)
2. Feed them into a sophisticated atmospheric model
3. The model fills gaps where no observations exist, using physics
4. Repeat for every hour going back to 1940 (or earlier)

**ERA5** is produced by the European Centre for Medium-Range Weather Forecasts (ECMWF). It's the gold standard for climate research:

| Feature | ERA5 |
|---------|------|
| Coverage | Global, 1940-present |
| Resolution | ~31 km, hourly |
| Variables | 100+ (temperature, precipitation, wind, humidity, etc.) |
| Updates | Near real-time (5-day lag) |

**Why reanalysis instead of raw station data?**
- **Complete coverage**: Data everywhere, including oceans and remote areas
- **Consistent methods**: Same processing for 1950 and 2024
- **Gap filling**: No missing days due to broken instruments

## Define area of interest: Missouri

We'll focus on Missouri to see local climate trends, but these methods work anywhere in the world.

In [ ]:
# Get Missouri boundary from TIGER
states = ee.FeatureCollection("TIGER/2018/States")
missouri = states.filter(ee.Filter.eq("NAME", "Missouri"))
mo_geometry = missouri.geometry()

# Center point for maps
center_lat = 38.5
center_lon = -92.5

print("Area of interest: Missouri")
print("We'll analyze climate trends from 1981-2024.")

## Load ERA5 Monthly Data

We'll use the ERA5-Land Monthly Aggregates dataset from Google Earth Engine. This dataset provides monthly means of key climate variables at ~11 km resolution.

**Key variables:**
- `temperature_2m`: Air temperature at 2 meters above surface (Kelvin)
- `total_precipitation_sum`: Total precipitation in the month (meters)

Note: ERA5-Land is higher resolution than standard ERA5 and goes back to 1950.

In [ ]:
# Load ERA5-Land Monthly Aggregated dataset
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")

# Check available date range
first = ee.Date(era5.first().get('system:time_start'))
last = ee.Date(era5.sort('system:time_start', False).first().get('system:time_start'))

print(f"ERA5-Land Monthly data available:")
print(f"  First: {first.format('YYYY-MM').getInfo()}")
print(f"  Last: {last.format('YYYY-MM').getInfo()}")
print(f"  Total images: {era5.size().getInfo()}")

## Temperature Analysis: Then vs Now

Let's compare average summer temperatures (June-August) between the 1980s and the most recent decade. Has Missouri gotten hotter?

We'll convert from Kelvin to Fahrenheit:
- °F = (K - 273.15) × 9/5 + 32

In [ ]:
def kelvin_to_fahrenheit(image):
    """Convert temperature from Kelvin to Fahrenheit."""
    return image.subtract(273.15).multiply(9/5).add(32)

# Summer months: June (6), July (7), August (8)
summer_months = [6, 7, 8]

# 1980s summer average (1981-1990)
era5_1980s = era5.filter(ee.Filter.calendarRange(1981, 1990, 'year'))
summer_1980s = era5_1980s.filter(ee.Filter.calendarRange(6, 8, 'month'))
temp_1980s = summer_1980s.select('temperature_2m').mean()
temp_1980s_f = kelvin_to_fahrenheit(temp_1980s).clip(mo_geometry)

# Recent decade summer average (2014-2023)
era5_recent = era5.filter(ee.Filter.calendarRange(2014, 2023, 'year'))
summer_recent = era5_recent.filter(ee.Filter.calendarRange(6, 8, 'month'))
temp_recent = summer_recent.select('temperature_2m').mean()
temp_recent_f = kelvin_to_fahrenheit(temp_recent).clip(mo_geometry)

# Calculate the difference
temp_diff = temp_recent_f.subtract(temp_1980s_f)

print("Computed summer temperature averages:")
print("  - 1980s (1981-1990)")
print("  - Recent (2014-2023)")

In [ ]:
# Temperature visualization palette (warm colors)
temp_palette = ['#313695', '#4575b4', '#74add1', '#abd9e9', 
                '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']

temp_vis = {
    'min': 70,
    'max': 85,
    'palette': temp_palette
}

# Create side-by-side comparison
left_layer = geemap.ee_tile_layer(temp_1980s_f, temp_vis, '1980s Summer Temp')
right_layer = geemap.ee_tile_layer(temp_recent_f, temp_vis, 'Recent Summer Temp')

m1 = geemap.Map(center=[center_lat, center_lon], zoom=7)
m1.split_map(left_layer, right_layer)

print("Left: 1980s average | Right: 2014-2023 average")
print("Look for the shift toward warmer colors (reds/oranges)")
m1

In [ ]:
# Map the temperature difference
diff_palette = ['#2166ac', '#67a9cf', '#d1e5f0', '#f7f7f7', 
                '#fddbc7', '#ef8a62', '#b2182b']

diff_vis = {
    'min': -2,
    'max': 2,
    'palette': diff_palette
}

m2 = geemap.Map(center=[center_lat, center_lon], zoom=7)
m2.addLayer(temp_diff, diff_vis, 'Temperature Change (°F)')
m2.addLayer(mo_geometry, {'color': 'black'}, 'Missouri', False)
m2.add_colorbar(diff_vis, label='Temperature Change (°F)', layer_name='Temperature Change (°F)')

print("Temperature change: Recent decade minus 1980s")
print("Red = warmer now | Blue = cooler now")
m2

## Time Series: Annual Temperature for Missouri

Maps show spatial patterns, but time series show the trend. Let's plot annual mean temperature for Missouri from 1981 to present.

We'll compute the mean temperature across all of Missouri for each year.

In [ ]:
def get_annual_mean_temp(year):
    """Get annual mean temperature for Missouri in a given year."""
    year = ee.Number(year)
    annual = era5.filter(ee.Filter.calendarRange(year, year, 'year'))
    temp_k = annual.select('temperature_2m').mean()
    temp_f = kelvin_to_fahrenheit(temp_k)
    
    # Get mean across Missouri
    mean_temp = temp_f.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=mo_geometry,
        scale=10000,  # ~10km for efficiency
        maxPixels=1e9
    ).get('temperature_2m')
    
    return ee.Feature(None, {'year': year, 'temp_f': mean_temp})

# Compute for years 1981-2024
years = list(range(1981, 2025))
print(f"Computing annual temperatures for {len(years)} years...")

# Create feature collection and extract data
temp_fc = ee.FeatureCollection([get_annual_mean_temp(y) for y in years])
temp_data = temp_fc.getInfo()

# Parse results
temp_years = []
temp_values = []
for f in temp_data['features']:
    props = f['properties']
    if props['temp_f'] is not None:
        temp_years.append(props['year'])
        temp_values.append(props['temp_f'])

print(f"Retrieved {len(temp_years)} years of data")

In [ ]:
# Plot temperature time series
fig, ax = plt.subplots(figsize=(12, 5))

# Plot annual values
ax.plot(temp_years, temp_values, 'o-', color='#d73027', markersize=4, linewidth=1, alpha=0.7)

# Add trend line
z = np.polyfit(temp_years, temp_values, 1)
p = np.poly1d(z)
ax.plot(temp_years, p(temp_years), '--', color='black', linewidth=2, label=f'Trend')

# Calculate warming rate
warming_per_year = z[0]
warming_per_decade = warming_per_year * 10

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Annual Mean Temperature (°F)', fontsize=12)
ax.set_title('Missouri Annual Mean Temperature (1981-2024)', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend()

# Add annotation
ax.text(0.02, 0.98, f'Warming rate: {warming_per_decade:.2f}°F per decade', 
        transform=ax.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\nWarming rate: {warming_per_decade:.2f}°F per decade ({warming_per_decade * 5/9:.2f}°C per decade)")

## Precipitation Analysis

Is Missouri getting wetter or drier? Let's look at annual precipitation trends.

ERA5 stores precipitation in **meters**. We'll convert to **inches** (more intuitive for US audience):
- 1 meter = 39.37 inches

In [ ]:
def meters_to_inches(image):
    """Convert precipitation from meters to inches."""
    return image.multiply(39.37)

def get_annual_precip(year):
    """Get total annual precipitation for Missouri in a given year."""
    year = ee.Number(year)
    annual = era5.filter(ee.Filter.calendarRange(year, year, 'year'))
    
    # Sum monthly precipitation totals
    precip_m = annual.select('total_precipitation_sum').sum()
    precip_in = meters_to_inches(precip_m)
    
    # Get mean across Missouri
    mean_precip = precip_in.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=mo_geometry,
        scale=10000,
        maxPixels=1e9
    ).get('total_precipitation_sum')
    
    return ee.Feature(None, {'year': year, 'precip_in': mean_precip})

# Compute for years 1981-2024
print(f"Computing annual precipitation for {len(years)} years...")

precip_fc = ee.FeatureCollection([get_annual_precip(y) for y in years])
precip_data = precip_fc.getInfo()

# Parse results
precip_years = []
precip_values = []
for f in precip_data['features']:
    props = f['properties']
    if props['precip_in'] is not None:
        precip_years.append(props['year'])
        precip_values.append(props['precip_in'])

print(f"Retrieved {len(precip_years)} years of data")

In [ ]:
# Plot precipitation time series
fig, ax = plt.subplots(figsize=(12, 5))

# Plot annual values as bars
colors = ['#2166ac' if v < np.mean(precip_values) else '#67a9cf' for v in precip_values]
ax.bar(precip_years, precip_values, color='#4575b4', alpha=0.7, edgecolor='none')

# Add mean line
mean_precip = np.mean(precip_values)
ax.axhline(mean_precip, color='black', linestyle='--', linewidth=2, label=f'Mean: {mean_precip:.1f} in')

# Add trend line
z_precip = np.polyfit(precip_years, precip_values, 1)
p_precip = np.poly1d(z_precip)
ax.plot(precip_years, p_precip(precip_years), '-', color='#d73027', linewidth=2, label='Trend')

precip_change_per_decade = z_precip[0] * 10

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Annual Precipitation (inches)', fontsize=12)
ax.set_title('Missouri Annual Precipitation (1981-2024)', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')
ax.legend(loc='upper left')

# Add annotation
ax.text(0.02, 0.88, f'Trend: {precip_change_per_decade:+.1f} inches per decade', 
        transform=ax.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\nPrecipitation trend: {precip_change_per_decade:+.2f} inches per decade")
print(f"Average annual precipitation: {mean_precip:.1f} inches")

In [ ]:
# Map average annual precipitation
annual_precip = era5.filter(ee.Filter.calendarRange(2014, 2023, 'year')).select('total_precipitation_sum')

# Sum by year, then average across years
def get_year_total(year):
    year = ee.Number(year)
    return era5.filter(ee.Filter.calendarRange(year, year, 'year')).select('total_precipitation_sum').sum()

yearly_totals = ee.ImageCollection([get_year_total(y) for y in range(2014, 2024)])
mean_annual = meters_to_inches(yearly_totals.mean()).clip(mo_geometry)

# Precipitation palette (blue scale)
precip_palette = ['#f7fbff', '#deebf7', '#c6dbef', '#9ecae1', 
                  '#6baed6', '#4292c6', '#2171b5', '#084594']

precip_vis = {
    'min': 35,
    'max': 55,
    'palette': precip_palette
}

m3 = geemap.Map(center=[center_lat, center_lon], zoom=7)
m3.addLayer(mean_annual, precip_vis, 'Mean Annual Precipitation (in)')
m3.addLayer(mo_geometry, {'color': 'black'}, 'Missouri', False)
m3.add_colorbar(precip_vis, label='Annual Precipitation (inches)', layer_name='Mean Annual Precipitation (in)')

print("Average annual precipitation (2014-2023)")
print("Southeast Missouri receives more precipitation than the northwest")
m3

## Seasonal Patterns: How Have Seasons Changed?

Climate change doesn't affect all seasons equally. Let's compare how summer (JJA) and winter (DJF) temperatures have changed.

In [ ]:
def get_seasonal_temp(year, season):
    """Get seasonal mean temperature for Missouri.
    season: 'winter' (DJF) or 'summer' (JJA)
    """
    year = ee.Number(year)
    
    if season == 'winter':
        # Winter = Dec of previous year + Jan, Feb of current year
        dec = era5.filter(ee.Filter.calendarRange(year.subtract(1), year.subtract(1), 'year')).filter(ee.Filter.eq('system:index', ee.Number(year.subtract(1)).format('%d').cat('12_01')))
        jan_feb = era5.filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(1, 2, 'month'))
        seasonal = jan_feb  # Simplified: just Jan-Feb
        seasonal = era5.filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(1, 2, 'month'))
    else:  # summer
        seasonal = era5.filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(6, 8, 'month'))
    
    temp_k = seasonal.select('temperature_2m').mean()
    temp_f = kelvin_to_fahrenheit(temp_k)
    
    mean_temp = temp_f.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=mo_geometry,
        scale=10000,
        maxPixels=1e9
    ).get('temperature_2m')
    
    return ee.Feature(None, {'year': year, 'temp_f': mean_temp})

print("Computing seasonal temperatures...")

# Get summer (JJA) temperatures
summer_fc = ee.FeatureCollection([get_seasonal_temp(y, 'summer') for y in years])
summer_data = summer_fc.getInfo()

summer_temps = []
summer_yrs = []
for f in summer_data['features']:
    props = f['properties']
    if props['temp_f'] is not None:
        summer_yrs.append(props['year'])
        summer_temps.append(props['temp_f'])

# Get winter (JF) temperatures
winter_fc = ee.FeatureCollection([get_seasonal_temp(y, 'winter') for y in years])
winter_data = winter_fc.getInfo()

winter_temps = []
winter_yrs = []
for f in winter_data['features']:
    props = f['properties']
    if props['temp_f'] is not None:
        winter_yrs.append(props['year'])
        winter_temps.append(props['temp_f'])

print(f"Retrieved summer data for {len(summer_temps)} years")
print(f"Retrieved winter data for {len(winter_temps)} years")

In [ ]:
# Plot seasonal comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Summer plot
ax1.plot(summer_yrs, summer_temps, 'o-', color='#d73027', markersize=4, linewidth=1, alpha=0.7)
z_summer = np.polyfit(summer_yrs, summer_temps, 1)
p_summer = np.poly1d(z_summer)
ax1.plot(summer_yrs, p_summer(summer_yrs), '--', color='black', linewidth=2)
summer_trend = z_summer[0] * 10

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Temperature (°F)', fontsize=12)
ax1.set_title('Summer (Jun-Aug) Temperature', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.text(0.02, 0.98, f'Trend: {summer_trend:+.2f}°F/decade', 
         transform=ax1.transAxes, fontsize=11, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))

# Winter plot
ax2.plot(winter_yrs, winter_temps, 'o-', color='#4575b4', markersize=4, linewidth=1, alpha=0.7)
z_winter = np.polyfit(winter_yrs, winter_temps, 1)
p_winter = np.poly1d(z_winter)
ax2.plot(winter_yrs, p_winter(winter_yrs), '--', color='black', linewidth=2)
winter_trend = z_winter[0] * 10

ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Temperature (°F)', fontsize=12)
ax2.set_title('Winter (Jan-Feb) Temperature', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.text(0.02, 0.98, f'Trend: {winter_trend:+.2f}°F/decade', 
         transform=ax2.transAxes, fontsize=11, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\nSummer warming: {summer_trend:.2f}°F per decade")
print(f"Winter warming: {winter_trend:.2f}°F per decade")

## Key Takeaways

**What we found for Missouri:**

1. **Temperature is rising**: The warming trend is clear in the data
   - Multiple lines of evidence: decade comparisons, time series, spatial maps
   
2. **Precipitation patterns**: Changes in precipitation are more variable than temperature
   - Year-to-year variation is large
   - Long-term trends are harder to detect

3. **Seasonal differences**: Climate change doesn't affect all seasons equally
   - Compare summer vs winter trends
   - This has implications for agriculture, ecosystems, and water resources

**Why reanalysis matters:**
- Consistent methodology over decades allows trend detection
- Complete spatial coverage fills gaps in station networks
- Multiple variables available for comprehensive analysis

**Limitations to keep in mind:**
- Reanalysis is a model product, not direct observations
- Accuracy varies by region and time period
- Earlier years may have fewer observations to constrain the model

## Try It Yourself

Ideas to explore:

1. **Other regions**: Change Missouri to another state or country. How does warming in the Midwest compare to the Southwest or Pacific Northwest?
   ```python
   # For other US states:
   state = states.filter(ee.Filter.eq("NAME", "Texas"))
   
   # For countries:
   countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
   france = countries.filter(ee.Filter.eq("country_na", "France"))
   ```

2. **Other variables**: ERA5 has many more variables to explore:
   - `snow_depth_water_equivalent`: Snowpack analysis
   - `u_component_of_wind_10m`, `v_component_of_wind_10m`: Wind patterns
   - `surface_solar_radiation_downwards_sum`: Solar radiation trends
   - `evaporation_from_vegetation_transpiration_sum`: Evapotranspiration

3. **Extreme years**: Identify the hottest/coldest, wettest/driest years. What weather patterns caused them?
   ```python
   # Find the warmest year
   warmest_year = temp_years[temp_values.index(max(temp_values))]
   ```

4. **Monthly analysis**: Instead of annual, look at trends for specific months. Is April warming faster than October?

5. **Compare to station data**: Download actual weather station data and compare to ERA5. How well does reanalysis match observations?

6. **Growing season length**: When does the last spring frost occur? First fall frost? Is the growing season getting longer?

7. **Drought analysis**: Combine temperature and precipitation to identify drought years. Use the Standardized Precipitation Index (SPI) or similar metrics.